# Importer les bibliothèques

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk

# Télécharger les stopwords français

In [2]:
nltk.download("stopwords")
from nltk.corpus import stopwords
french_stopwords = stopwords.words("french")

[nltk_data] Downloading package stopwords to C:\Users\Galaye
[nltk_data]     MBENGUE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Charger et combiner les données des deux fichiers CSV

In [3]:
# Charger les données 
fightclub_df = pd.read_csv("fightclub_critiques.csv")
interstellar_df = pd.read_csv("interstellar_critiques.csv")

# Ajouter une colonne pour identifier le film
fightclub_df["movie_title"] = "Fight Club"
interstellar_df["movie_title"] = "Interstellar"

# Combiner les deux films dans un seul DataFrame
df = pd.concat([fightclub_df, interstellar_df], ignore_index=True)
df

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316,Fight Club
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552,Fight Club
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207,Fight Club
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333,Fight Club
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721,Fight Club
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,62544336,https://senscritique.com/film/xxx/critique/625...,9,23/07/15 16:11,23/07/15 16:11,245,<p>Courez le voir ! </p>\n\n<p>C'est pour moi ...,Déjà culte !,0,Cirufufu,257665,Interstellar
1996,115084516,https://senscritique.com/film/xxx/critique/115...,8,31/12/16 18:29,31/12/16 18:29,245,"<p>Des effets spéciaux magnifiques, des paysag...",Bluffant mais...,0,Popi22,526273,Interstellar
1997,165773873,https://senscritique.com/film/xxx/critique/165...,10,18/04/18 12:40,18/04/18 12:40,244,<p>Quel est le rôle de la science fiction ? Qu...,Resentir le vide pour un fois !,1,TristanBellair,677796,Interstellar
1998,84414025,https://senscritique.com/film/xxx/critique/844...,10,25/03/17 00:47,14/08/22 10:03,244,<p>Christopher Nolan nous livre encore une pép...,Troublant et excellent,0,Revan2107,551260,Interstellar


# prétraitement des données textuelles

In [4]:
df.isnull().sum()

id                           0
URL                          0
rating                       0
review_date_creation         0
review_date_last_update      0
review_hits                  0
review_content               1
review_title               519
gen_review_like_count        0
username                     0
user_id                      0
movie_title                  0
dtype: int64

In [5]:
# Supprimer les lignes où review_content est NaN
df = df.dropna(subset=["review_content"]).reset_index(drop=True)
df

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316,Fight Club
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552,Fight Club
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207,Fight Club
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333,Fight Club
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721,Fight Club
...,...,...,...,...,...,...,...,...,...,...,...,...
1994,62544336,https://senscritique.com/film/xxx/critique/625...,9,23/07/15 16:11,23/07/15 16:11,245,<p>Courez le voir ! </p>\n\n<p>C'est pour moi ...,Déjà culte !,0,Cirufufu,257665,Interstellar
1995,115084516,https://senscritique.com/film/xxx/critique/115...,8,31/12/16 18:29,31/12/16 18:29,245,"<p>Des effets spéciaux magnifiques, des paysag...",Bluffant mais...,0,Popi22,526273,Interstellar
1996,165773873,https://senscritique.com/film/xxx/critique/165...,10,18/04/18 12:40,18/04/18 12:40,244,<p>Quel est le rôle de la science fiction ? Qu...,Resentir le vide pour un fois !,1,TristanBellair,677796,Interstellar
1997,84414025,https://senscritique.com/film/xxx/critique/844...,10,25/03/17 00:47,14/08/22 10:03,244,<p>Christopher Nolan nous livre encore une pép...,Troublant et excellent,0,Revan2107,551260,Interstellar


### Nettoyage du texte

In [6]:
def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)  # retire les balises HTML
    text = text.lower()
    return text

# Appliquer la fonction de nettoyage clean_text sur toutes les critiques
# et stocker le texte nettoyé dans une nouvelle colonne 'clean_review'

df["clean_review"] = df["review_content"].apply(clean_text)
df

,id,URL,rating,review_date_creation,review_date_last_update,review_hits,review_content,review_title,gen_review_like_count,username,user_id,movie_title,clean_review
0,20761,https://senscritique.com/film/xxx/critique/20761,10,12/03/10 00:00,12/03/10 00:00,35817,Fight Club sort à la fin 1999 sur les écrans. ...,Analyse du film et du livre,292,petaire,316,Fight Club,fight club sort à la fin 1999 sur les écrans. ...
1,30753423,https://senscritique.com/film/xxx/critique/307...,8,28/01/15 09:33,28/01/15 09:33,19867,Tout a été déjà dit sur le film de David Finch...,La consommation identitaire,163,Velvetman,178552,Fight Club,tout a été déjà dit sur le film de david finch...
2,12057488,https://senscritique.com/film/xxx/critique/120...,10,19/07/15 02:55,19/07/15 02:55,12672,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",Sons of Anarchy.,199,Gand-Alf,55207,Fight Club,"qu'une oeuvre aussi folle, aussi inconfortable..."
3,10939046,https://senscritique.com/film/xxx/critique/109...,5,05/02/12 14:40,19/08/12 03:33,12669,<p>Beaucoup pensent que ce film n'est qu'un él...,Je suis l'ego démesuré de Jack,191,Ano,2333,Fight Club,beaucoup pensent que ce film n'est qu'un éloge...
4,1966438,https://senscritique.com/film/xxx/critique/196...,9,27/08/14 03:18,27/08/14 03:18,7677,"<p>Objet d’un véritable culte, Fight Club a ma...",Il est temps d'enfreindre les deux premières r...,135,GagReathle,4721,Fight Club,"objet d’un véritable culte, fight club a marqu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,62544336,https://senscritique.com/film/xxx/critique/625...,9,23/07/15 16:11,23/07/15 16:11,245,<p>Courez le voir ! </p>\n\n<p>C'est pour moi ...,Déjà culte !,0,Cirufufu,257665,Interstellar,courez le voir ! \n\nc'est pour moi le 2001 : ...
1995,115084516,https://senscritique.com/film/xxx/critique/115...,8,31/12/16 18:29,31/12/16 18:29,245,"<p>Des effets spéciaux magnifiques, des paysag...",Bluffant mais...,0,Popi22,526273,Interstellar,"des effets spéciaux magnifiques, des paysages ..."
1996,165773873,https://senscritique.com/film/xxx/critique/165...,10,18/04/18 12:40,18/04/18 12:40,244,<p>Quel est le rôle de la science fiction ? Qu...,Resentir le vide pour un fois !,1,TristanBellair,677796,Interstellar,quel est le rôle de la science fiction ? quell...
1997,84414025,https://senscritique.com/film/xxx/critique/844...,10,25/03/17 00:47,14/08/22 10:03,244,<p>Christopher Nolan nous livre encore une pép...,Troublant et excellent,0,Revan2107,551260,Interstellar,christopher nolan nous livre encore une pépite...


### Calculer TF-IDF

In [7]:
tfidf_vectorizer = TfidfVectorizer(stop_words=french_stopwords)
tfidf_matrix = tfidf_vectorizer.fit_transform(df["clean_review"])

print("TF-IDF calculé pour", tfidf_matrix.shape[0], "critiques et", tfidf_matrix.shape[1], "mots/features.")

TF-IDF calculé pour 1999 critiques et 28233 mots/features.


# Fonction : récupérer les critiques les plus similaires à une critique donnée


In [8]:
def get_similar_reviews(critique_id, top_n=5, use_rating=False, rating_weight=0.5):
    """
    Récupère les top N critiques les plus similaires à une critique donnée,
    avec possibilité de prendre en compte le rating et le nombre de likes.

    Paramètres :
    - critique_id : ID de la critique de référence
    - top_n : nombre de critiques à retourner
    - use_rating : si True, pondère la similarité selon la note
    - rating_weight : poids de la note dans le score combiné (0 à 1)
    """
    # Vérifier que l'ID existe
    if critique_id not in df["id"].values:
        return f"Critique {critique_id} introuvable"
    
    # Récupérer l'indice de la critique
    idx = df.index[df["id"] == critique_id][0]
    movie = df.loc[idx, "movie_title"]
    rating_ref = df.loc[idx, "rating"]
    
    # Filtrer les critiques du même film
    movie_indices = df[df["movie_title"] == movie].index
    movie_tfidf = tfidf_matrix[movie_indices]
    
    # Calculer la similarité cosine
    sim_scores = cosine_similarity(tfidf_matrix[idx], movie_tfidf).flatten()
    
    # Exclure la critique elle-même
    sim_scores[df.index.get_indexer([idx])] = 0
    
    # Pondération par rating
    if use_rating:
        rating_diff = np.abs(df.loc[movie_indices, "rating"] - rating_ref)
        rating_score = 1 - (rating_diff / df["rating"].max())  # Normaliser entre 0 et 1
        sim_scores = (1 - rating_weight) * sim_scores + rating_weight * rating_score
    
    # Trier indices par score combiné
    top_indices = sim_scores.argsort()[::-1][:top_n]
    
    # Récupérer la critique d’entrée
    current_review = df.loc[[idx], ["id", "review_title", "review_content", "rating", "gen_review_like_count"]].copy()
    current_review["similarity"] = 1.0  # Similarité maximale avec elle-même
    
    # Récupérer les critiques similaires
    similar_reviews = df.iloc[movie_indices[top_indices]][
        ["id", "review_title", "review_content", "rating", "gen_review_like_count"]
    ].copy()
    similar_reviews["similarity"] = sim_scores[top_indices]
    
    # Trier secondairement par nombre de likes
    similar_reviews = similar_reviews.sort_values(by=["similarity", "gen_review_like_count"], ascending=[False, False])
    
    # Concaténer : critique d'entrée + résultats
    result = pd.concat([current_review, similar_reviews], ignore_index=True)
    
    return result.reset_index(drop=True)


### Exemple à tester 

In [9]:
# Critique de référence : 
critique_id = df[df["movie_title"] == "Fight Club"]["id"].iloc[2]

# Top 5 critiques similaires en prenant en compte le rating
top_similaires = get_similar_reviews(critique_id, top_n=5, use_rating=True, rating_weight=0.3)

top_similaires

,id,review_title,review_content,rating,gen_review_like_count,similarity
0,12057488,Sons of Anarchy.,"<p>Qu'une oeuvre aussi folle, aussi inconforta...",10,199,1.000000
1,6092115,Avec le savon on ne fait que du propre !,"On pourra dire ce qu'on veut, et même si ce dé...",10,1,0.418821
2,61075889,Club privé. Pour hommes uniquement.,<pre><code> HA ! HA ! HA ! HAAAA ! HI! HI! HI...,10,3,0.413709
3,41298149,Faut pas le prendre pour ce qu'il n'est pas,<p>[Grosse refonte du 08/11/2016]\nJ'ai envie ...,9,14,0.412559
4,97712264,"L’armée des 12 singes, sinon plus...","<p>Les années passent, les gouvernements se su...",10,7,0.410382
5,21701678,CRITIQUE et ANALYSE personnelles dans le détai...,David Fincher et son grand duo d'acteurs (Brad...,9,8,0.409869


In [10]:
# Afficher le texte complet dans toutes les colonnes
pd.set_option('display.max_colwidth', None)

# Maintenant afficher le DataFrame
top_similaires


id  \
0  12057488   
1   6092115   
2  61075889   
3  41298149   
4  97712264   
5  21701678   

                                                                                           review_title  \
0                                                                                      Sons of Anarchy.   
1                                                              Avec le savon on ne fait que du propre !   
2                                                                   Club privé. Pour hommes uniquement.   
3                                                           Faut pas le prendre pour ce qu'il n'est pas   
4                                                                  L’armée des 12 singes, sinon plus...   
5  CRITIQUE et ANALYSE personnelles dans le détail du début jusqu'à la fin du film. Attention Spoilers!   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 